# **Libraries**


In [ ]:
import pandas as pd
import numpy as np
import collections

import spacy

import nltk


nltk.download("all", quiet=True)

from nltk.corpus import stopwords

from nltk import word_tokenize

import string

from nltk.stem import WordNetLemmatizer


import wordcloud

import matplotlib.pyplot as plt

# **Reading Data**


In [3]:
raw_data = pd.read_csv("../data/raw_data.csv")
raw_data.head()

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NaN,0.11,1,0,0,0,0,0
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NaN,0.29,1,0,0,0,0,0
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NaN,0.51,1,0,0,0,0,0
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NaN,0.32,1,0,0,0,0,0
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NaN,0.80,1,0,0,0,0,0


# **Quick Data Understanding**


Data Types


In [4]:
raw_data.dtypes

element_id              object
site_id                 object
seller_id                int64
Dominio_normalizado     object
Titulo                  object
Marca                   object
Precio                 float64
Rule                    object
Score                  float64
Total                    int64
Moderado                 int64
Fake                     int64
Rollback                 int64
FK_TEST1                 int64
FK_TEST2                 int64
dtype: object

Duplicated Vaues


In [5]:
raw_data.duplicated().sum()

0

Nulls


In [6]:
raw_data.isna().sum()
pd.DataFrame(raw_data.isna().sum() / len(raw_data) * 100).sort_values(
    0, ascending=False
)

,0
Rule,89.403547
element_id,0.000000
site_id,0.000000
seller_id,0.000000
Dominio_normalizado,0.000000
Titulo,0.000000
Marca,0.000000
Precio,0.000000
Score,0.000000
Total,0.000000


Review Some Distributions


In [7]:
raw_data.describe()

,seller_id,Precio,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
count,4.037200e+04,4.037200e+04,40372.000000,40372.0,40372.000000,40372.000000,40372.000000,40372.000000,40372.000000
mean,1.911603e+09,2.455866e+05,0.428654,1.0,0.105965,0.111191,0.007580,0.047483,0.007282
std,1.784332e+09,6.435465e+05,0.330362,0.0,0.307796,0.314372,0.086731,0.212673,0.085026
min,4.281520e+05,1.117000e+01,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.440346e+08,6.767050e+02,0.150000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.217641e+09,4.299000e+03,0.310000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.609587e+09,1.549900e+05,0.810000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.880437e+09,1.289990e+07,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


# **Building Final Dataset**


In [8]:
process_data = raw_data.copy()
process_data["Rule"] = process_data["Rule"].fillna("NOT_MODERATED")
process_data

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NOT_MODERATED,0.11,1,0,0,0,0,0
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NOT_MODERATED,0.29,1,0,0,0,0,0
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NOT_MODERATED,0.51,1,0,0,0,0,0
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NOT_MODERATED,0.32,1,0,0,0,0,0
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NOT_MODERATED,0.80,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40367,PER9979201495,PERU,1906413812,SPEAKERS,Bocina Jbl Charge 5 Jblcharge5 Portátil Con Bl...,JBL,799.0,NOT_MODERATED,0.13,1,0,0,0,0,0
40368,PER9979619481,PERU,699068980,HEADPHONES,Audifonos Jbl Tune 520 Bt Bluetooth On Ear Col...,JBL,299.0,NOT_MODERATED,0.80,1,0,0,0,0,0
40369,PER9980515950,PERU,1510316564,SMARTWATCHES,Apple Watch Series 10 Gps Caja De Aluminio Ne...,Apple,2499.0,NOT_MODERATED,0.83,1,0,0,0,0,0
40370,PER999421010,PERU,2495083952,WRISTWATCHES,Reloj Casio W218wd Acero Inoxidable Alarma Sum...,Casio,389.0,NOT_MODERATED,0.10,1,0,0,0,0,0


# **Building Text Dataset**


In [ ]:
nlp = spacy.load("es_core_news_sm")
nlp = spacy.load("pt_core_news_sm")

In [33]:
nlp_p = spacy.load("pt_core_news_sm")
nlp_s = spacy.load("es_core_news_sm")
def non_lexical_features_p(text: str, language: str) -> float:
    doc = nlp_p(text)
    words = [
        token.text.lower()
        for token in doc
        if token.is_alpha and not token.is_stop and not token.is_punct
    ]
    tot_words = len(words)
    if tot_words == 0:
        return 0, 0, 0
    else:
        sentences = nltk.tokenize.sent_tokenize(text, language=language)
        average_length = sum(len(sentence.split()) for sentence in sentences) / len(
        sentences
    )
        unique_verbs = set(
            [
                token.text.lower()
                for token in nlp(" ".join(words))
                if token.pos_ == "VERB"
            ]
        )
        unique_nouns = set(
            [
                token.text.lower()
                for token in nlp(" ".join(words))
                if token.pos_ == "NOUN"
            ]
        )
        unique_adjetives = set(
            [
                token.text.lower()
                for token in nlp(" ".join(words))
                if token.pos_ == "ADJ"
            ]
        )
        return (
            len(sentences),
            average_length,
            np.round(len(unique_verbs) / tot_words, 4),
            np.round(len(unique_nouns) / tot_words, 4),
            np.round(len(unique_adjetives) / tot_words, 4),
        )

def non_lexical_features_s(text: str, language: str) -> float:
    doc = nlp_s(text)
    words = [
        token.text.lower()
        for token in doc
        if token.is_alpha and not token.is_stop and not token.is_punct
    ]
    tot_words = len(words)
    if tot_words == 0:
        return 0, 0, 0
    else:
        sentences = nltk.tokenize.sent_tokenize(text, language=language)
        average_length = sum(len(sentence.split()) for sentence in sentences) / len(
        sentences
    )
        unique_verbs = set(
            [
                token.text.lower()
                for token in nlp(" ".join(words))
                if token.pos_ == "VERB"
            ]
        )
        unique_nouns = set(
            [
                token.text.lower()
                for token in nlp(" ".join(words))
                if token.pos_ == "NOUN"
            ]
        )
        unique_adjetives = set(
            [
                token.text.lower()
                for token in nlp(" ".join(words))
                if token.pos_ == "ADJ"
            ]
        )
        return (
            len(sentences),
            average_length,
            np.round(len(unique_verbs) / tot_words, 4),
            np.round(len(unique_nouns) / tot_words, 4),
            np.round(len(unique_adjetives) / tot_words, 4),
        )

Normalizing Data


In [39]:
def preprocess(text: str, language: str) -> tuple:
    stop_words = set(stopwords.words(language))
    wordnet_lemmatizer = WordNetLemmatizer()
    text = text.lower()
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    lemas = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemas), len(lemas)


text_brasil = (
    process_data[process_data["site_id"] == "BRASIL"]["Titulo"]
    .apply(lambda x: preprocess(x, "portuguese"))
    .apply(pd.Series)
    .rename(columns={0: "normalized", 1: "number_of_words_no_stopwords"})
)
text_spanish = (
    process_data[process_data["site_id"] != "BRASIL"]["Titulo"]
    .apply(lambda x: preprocess(x, "spanish"))
    .apply(pd.Series)
    .rename(columns={0: "normalized", 1: "number_of_words_no_stopwords"})
)
text_all_countries = pd.concat([text_spanish, text_brasil], axis=0).sort_index()
df_final_processed_data = pd.concat([process_data, text_all_countries], axis=1)

In [ ]:
text_brasil_2 = text_brasil.apply(
    lambda x: non_lexical_features_p(x["normalized"], "portuguese"),
    axis=1,
).apply(pd.Series)
text_brasil_2.columns = [
    "number_of_sentences",
    "average_length",
    "unique_verbs",
    "unique_nouns",
    "unique_adjetives",
]
text_brasil_2

,number_of_sentences,average_length,unique_verbs,unique_nouns,unique_adjetives
11938,1.0,8.0,0.0000,0.1250,0.0000
11939,1.0,9.0,0.0000,0.3750,0.2500
11940,1.0,9.0,0.2857,0.1429,0.4286
11941,1.0,6.0,0.2000,0.2000,0.2000
11942,1.0,10.0,0.0000,0.2222,0.1111
...,...,...,...,...,...
26075,1.0,9.0,0.1250,0.5000,0.1250
26076,1.0,7.0,0.6000,0.2000,0.0000
26077,1.0,10.0,0.2500,0.2500,0.1250
26078,1.0,8.0,0.1429,0.1429,0.2857


In [34]:
text_spanish_2 = text_spanish.apply(
    lambda x: non_lexical_features_s(x["normalized"], "spanish"),
    axis=1,
).apply(pd.Series)
text_spanish_2.columns = [
    "number_of_sentences",
    "average_length",
    "unique_verbs",
    "unique_nouns",
    "unique_adjetives",
]
text_spanish_2

,number_of_sentences,average_length,unique_verbs,unique_nouns,unique_adjetives
0,1.0,23.0,0.1667,0.0833,0.5000
1,1.0,10.0,0.0000,0.4286,0.2857
2,1.0,13.0,0.1538,0.3077,0.3077
3,1.0,10.0,0.1429,0.1429,0.2857
4,1.0,9.0,0.2857,0.2857,0.1429
...,...,...,...,...,...
40367,1.0,10.0,0.1429,0.2857,0.0000
40368,1.0,10.0,0.2222,0.2222,0.1111
40369,1.0,15.0,0.1538,0.1538,0.3077
40370,1.0,11.0,0.1000,0.2000,0.2000


In [40]:
text_all_non_lexical = pd.concat([text_spanish_2, text_brasil_2], axis=0).sort_index()
df_final_processed_data = pd.concat([df_final_processed_data, text_all_non_lexical], axis=1)
df_final_processed_data

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Total,...,Rollback,FK_TEST1,FK_TEST2,normalized,number_of_words_no_stopwords,number_of_sentences,average_length,unique_verbs,unique_nouns,unique_adjetives
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NOT_MODERATED,0.11,1,...,0,0,0,microprocesador intel core i3 12100 12mb bx807...,23,1.0,23.0,0.1667,0.0833,0.5000
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NOT_MODERATED,0.29,1,...,0,0,0,monitor gamer samsung pantalla 25 60hz 220v re...,10,1.0,10.0,0.0000,0.4286,0.2857
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NOT_MODERATED,0.51,1,...,0,0,0,anteojos sol polarizados rayban erika classic ...,13,1.0,13.0,0.1538,0.3077,0.3077
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NOT_MODERATED,0.32,1,...,0,0,0,ssd externo kingston xs1000 1tb negro usb 32 g...,10,1.0,10.0,0.1429,0.1429,0.2857
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NOT_MODERATED,0.80,1,...,0,0,0,memoria ram valueram color verde 8gb 1 kingsto...,9,1.0,9.0,0.2857,0.2857,0.1429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40367,PER9979201495,PERU,1906413812,SPEAKERS,Bocina Jbl Charge 5 Jblcharge5 Portátil Con Bl...,JBL,799.0,NOT_MODERATED,0.13,1,...,0,0,0,bocina jbl charge 5 jblcharge5 portátil blueto...,10,1.0,10.0,0.1429,0.2857,0.0000
40368,PER9979619481,PERU,699068980,HEADPHONES,Audifonos Jbl Tune 520 Bt Bluetooth On Ear Col...,JBL,299.0,NOT_MODERATED,0.80,1,...,0,0,0,audifonos jbl tune 520 bt bluetooth on ear col...,10,1.0,10.0,0.2222,0.2222,0.1111
40369,PER9980515950,PERU,1510316564,SMARTWATCHES,Apple Watch Series 10 Gps Caja De Aluminio Ne...,Apple,2499.0,NOT_MODERATED,0.83,1,...,0,0,0,apple watch series 10 gps caja aluminio negro ...,15,1.0,15.0,0.1538,0.1538,0.3077
40370,PER999421010,PERU,2495083952,WRISTWATCHES,Reloj Casio W218wd Acero Inoxidable Alarma Sum...,Casio,389.0,NOT_MODERATED,0.10,1,...,0,0,0,reloj casio w218wd acero inoxidable alarma sum...,11,1.0,11.0,0.1000,0.2000,0.2000


In [41]:
df_final_processed_data.to_csv("../data/processed_data.csv", index=False)